In [1]:
# import packages for data wrangling
import pandas as pd
from pandas.api.types import CategoricalDtype
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
# import csv file into a data frame for encoding ordinal and nominal variables
df = pd.read_csv('./Documents/WGU/D599/Task 3/Megastore_Dataset_Task_3 3.csv')

In [3]:
# C1a. identify the unique values for each categorical variables
for col in ["Country", "DiscountApplied", "OrderPriority", "Region", "Segment", "ExpeditedShipping", "PaymentMethod", "CustomerOrderSatisfaction"]:
    df[col] = df[col].astype(str).str.strip()

    print(f"{col}: {df[col].unique()}")

Country: ['United States']
DiscountApplied: ['Yes' 'No']
OrderPriority: ['High' 'Medium']
Region: ['Northeast' 'Southeast']
Segment: ['Corporate' 'Consumer']
ExpeditedShipping: ['Yes' 'No']
PaymentMethod: ['Credit Card' 'PayPal']
CustomerOrderSatisfaction: ['Satisfied' 'Dissatisfied' 'Prefer not to answer' 'Very Satisfied'
 'Very Dissatisfied']


In [4]:
# C1b. encode ordinal variables using ordinal encoding: OrderPriority and CustomerOrderSatisfaction
# Define custom orderings
priority_order = ["Medium", "High"]
satisfaction_order = [
    "Very Dissatisfied", "Dissatisfied",
    "Prefer not to answer", "Satisfied", "Very Satisfied"
]

# Create categorical dtypes with ordering
priority_dtype = CategoricalDtype(categories=priority_order, ordered=True)
satisfaction_dtype = CategoricalDtype(categories=satisfaction_order, ordered=True)

# apply to data frame
df["OrderPriority"] = df["OrderPriority"].astype(priority_dtype)
df["CustomerOrderSatisfaction"] = df["CustomerOrderSatisfaction"].astype(satisfaction_dtype)

# Encode as integer codes
df["OrderPriority_encoded"] = df["OrderPriority"].cat.codes
df["CustomerSatisfaction_encoded"] = df["CustomerOrderSatisfaction"].cat.codes
df.head(5)

,OrderID,ProductName,Quantity,InvoiceDate,UnitPrice,TotalCost,Country,DiscountApplied,OrderPriority,Region,Segment,ExpeditedShipping,PaymentMethod,CustomerOrderSatisfaction,OrderPriority_encoded,CustomerSatisfaction_encoded
0,536370,INFLATABLE POLITICAL GLOBE,48,12/1/2010 8:45,$0.85,$40.80,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied,1,3
1,536370,SET2 RED RETROSPOT TEA TOWELS,18,12/1/2010 8:45,$2.95,$53.10,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied,1,3
2,536370,PANDA AND BUNNIES STICKER SHEET,12,12/1/2010 8:45,$0.85,$10.20,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied,1,3
3,536370,RED TOADSTOOL LED NIGHT LIGHT,24,12/1/2010 8:45,$1.65,$39.60,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied,1,3
4,536370,VINTAGE HEADS AND TAILS CARD GAME,24,12/1/2010 8:45,$1.25,$30.00,United States,Yes,High,Northeast,Corporate,Yes,Credit Card,Satisfied,1,3


In [5]:
# C1b. encode nominal variables using one-hot encoding: Region and Segment
df = pd.get_dummies(df, columns=["Region", "Segment"], drop_first=False, dtype="int8")
df.head(5)

,OrderID,ProductName,Quantity,InvoiceDate,UnitPrice,TotalCost,Country,DiscountApplied,OrderPriority,ExpeditedShipping,PaymentMethod,CustomerOrderSatisfaction,OrderPriority_encoded,CustomerSatisfaction_encoded,Region_Northeast,Region_Southeast,Segment_Consumer,Segment_Corporate
0,536370,INFLATABLE POLITICAL GLOBE,48,12/1/2010 8:45,$0.85,$40.80,United States,Yes,High,Yes,Credit Card,Satisfied,1,3,1,0,0,1
1,536370,SET2 RED RETROSPOT TEA TOWELS,18,12/1/2010 8:45,$2.95,$53.10,United States,Yes,High,Yes,Credit Card,Satisfied,1,3,1,0,0,1
2,536370,PANDA AND BUNNIES STICKER SHEET,12,12/1/2010 8:45,$0.85,$10.20,United States,Yes,High,Yes,Credit Card,Satisfied,1,3,1,0,0,1
3,536370,RED TOADSTOOL LED NIGHT LIGHT,24,12/1/2010 8:45,$1.65,$39.60,United States,Yes,High,Yes,Credit Card,Satisfied,1,3,1,0,0,1
4,536370,VINTAGE HEADS AND TAILS CARD GAME,24,12/1/2010 8:45,$1.25,$30.00,United States,Yes,High,Yes,Credit Card,Satisfied,1,3,1,0,0,1


In [ ]:
# C1b. export new data frame with encoded variables to csv file
df.to_csv("EncodedVariables.csv", index=False)

In [6]:
# C1c. import csv file into a data frame for market basket analysis
df = pd.read_csv('/Users/ayu/Documents/WGU/D599/Task 3/Megastore_Dataset_Task_3 3.csv')

In [7]:
# C1c. Transactionalize the data for market basket analysis by grouping OrderID and aggregate ProductName into lists
basket = df.groupby("OrderID")["ProductName"].apply(list)
basket.head(5)

OrderID
536370    [INFLATABLE POLITICAL GLOBE , SET2 RED RETROSP...
536852    [POLKADOT RAIN HAT , VINTAGE HEADS AND TAILS C...
536974    [EDWARDIAN PARASOL RED, LUNCH BAG RED RETROSPO...
537065    [PARTY TIME PENCIL ERASERS, RED RETROSPOT PURS...
537463    [PINK POLKADOT CHILDRENS UMBRELLA, RED RETROSP...
Name: ProductName, dtype: object

In [8]:
# C1c. Transactionalize the data for market basket by transforming basket into mathematical table to then calculate probabilities
te = TransactionEncoder()
te_ary = te.fit(basket).transform(basket)
basket_df = pd.DataFrame(te_ary, columns=te.columns_)
basket_df

,50S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR TLIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC TLIGHT HOLDER STAR LARGE,ZINC TLIGHT HOLDER STARS SMALL
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
437,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
438,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
439,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# C2. export data frame after transactionalizing data to csv file
basket_df.to_csv("MarketBasketAnalysis.csv", index=False)

In [9]:
# C3. apply apriori algorithm to find frequent itemsets - only show combination of itemsets appear in at least 1% of transactions
frequent_itemsets = apriori(basket_df, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.020408,( DOLLY GIRL BEAKER)
1,0.011338,( I LOVE LONDON MINI BACKPACK)
2,0.013605,( SET 2 TEA TOWELS I LOVE LONDON )
3,0.036281,( SPACEBOY BABY GIFT SET)
4,0.027211,(10 COLOUR SPACEBOY PEN)
...,...,...
8335,0.011338,"(SET OF 9 BLACK SKULL BALLOONS, SET OF 9 HEART..."
8336,0.011338,"(SET OF 9 BLACK SKULL BALLOONS, SET OF 9 HEART..."
8337,0.011338,"(SET OF 9 BLACK SKULL BALLOONS, SET OF 9 HEART..."
8338,0.011338,"(ALARM CLOCK BAKELIKE GREEN, PLASTERS IN TIN S..."


In [10]:
# C4. provide values for the support, lift, and confidence of the association rules table
raw_rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.3)
rules = raw_rules[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'lift']].sort_values(by="lift", ascending=False)
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
84582,"(PLASTERS IN TIN CIRCUS PARADE , SET6 RED SPOT...","(ALARM CLOCK BAKELIKE RED , SET6 RED SPOTTY PA...",0.011338,0.011338,0.011338,1.0,88.2
82787,"(CARD DOLLY GIRL , ROUND SNACK BOXES SET OF4 W...","(ALARM CLOCK BAKELIKE RED , CHILDRENS CUTLERY ...",0.011338,0.011338,0.011338,1.0,88.2
55438,"(CARD DOLLY GIRL , ALARM CLOCK BAKELIKE PINK)","(ALARM CLOCK BAKELIKE RED , SPACEBOY BIRTHDAY ...",0.011338,0.011338,0.011338,1.0,88.2
82786,"(ALARM CLOCK BAKELIKE RED , CARD DOLLY GIRL , ...","(ALARM CLOCK BAKELIKE PINK, CHILDRENS CUTLERY ...",0.011338,0.011338,0.011338,1.0,88.2
80473,"(ALARM CLOCK BAKELIKE RED , ALARM CLOCK BAKELI...","(PLASTERS IN TIN CIRCUS PARADE , CHARLOTTE BAG...",0.011338,0.011338,0.011338,1.0,88.2


In [11]:
# C5. identify top three relevant rules generated by the Apriori algorithm
top_rules = (
    rules.sort_values(by="lift", ascending=False)   
         .head(3)                                   
)

for idx, row in top_rules.iterrows():
    print(f"Rule {idx + 1}: If a customer buys {set(row['antecedents'])}, "
          f"they also tend to buy {set(row['consequents'])}.")
    print(f"  • Support   : {row['support']:.3f}")
    print(f"  • Confidence: {row['confidence']:.3f}")
    print(f"  • Lift      : {row['lift']:.2f}\n") 


Rule 84583: If a customer buys {'PLASTERS IN TIN CIRCUS PARADE ', 'SET6 RED SPOTTY PAPER PLATES', 'ROUND SNACK BOXES SET OF4 WOODLAND ', 'ALARM CLOCK BAKELIKE GREEN'}, they also tend to buy {'ALARM CLOCK BAKELIKE RED ', 'SET6 RED SPOTTY PAPER CUPS', 'PLASTERS IN TIN SPACEBOY', 'ALARM CLOCK BAKELIKE PINK'}.
  • Support   : 0.011
  • Confidence: 1.000
  • Lift      : 88.20

Rule 84566: If a customer buys {'ROUND SNACK BOXES SET OF4 WOODLAND ', 'SET6 RED SPOTTY PAPER PLATES', 'PLASTERS IN TIN SPACEBOY', 'ALARM CLOCK BAKELIKE GREEN'}, they also tend to buy {'ALARM CLOCK BAKELIKE RED ', 'SET6 RED SPOTTY PAPER CUPS', 'PLASTERS IN TIN CIRCUS PARADE ', 'ALARM CLOCK BAKELIKE PINK'}.
  • Support   : 0.011
  • Confidence: 1.000
  • Lift      : 88.20

Rule 73457: If a customer buys {'SET6 RED SPOTTY PAPER PLATES', 'ROUND SNACK BOXES SET OF4 WOODLAND ', 'ALARM CLOCK BAKELIKE GREEN'}, they also tend to buy {'ALARM CLOCK BAKELIKE RED ', 'SET6 RED SPOTTY PAPER CUPS', 'PLASTERS IN TIN CIRCUS PARADE '}.